In [1]:
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

In [2]:
""" pdf_loader = DirectoryLoader('../../training_data/', glob="**/*.pdf")
txt_loader = DirectoryLoader('../../training_data/', glob="**/*.txt")
word_loader = DirectoryLoader('../../training_data/', glob="**/*.docx")
csv_loader = DirectoryLoader('../../training_data/', glob="**/*.csv") """

' pdf_loader = DirectoryLoader(\'../../training_data/\', glob="**/*.pdf")\ntxt_loader = DirectoryLoader(\'../../training_data/\', glob="**/*.txt")\nword_loader = DirectoryLoader(\'../../training_data/\', glob="**/*.docx")\ncsv_loader = DirectoryLoader(\'../../training_data/\', glob="**/*.csv") '

In [3]:
from bs4 import BeautifulSoup as Soup

url = "https://developers.hubspot.com/docs/cms/hubl"
recursive_url_loader = RecursiveUrlLoader(url=url, max_depth=200, extractor=lambda x: Soup(x, "html.parser").text)

loaders = [recursive_url_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")

Total number of documents: 34


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
huggingfacehub_api_key= os.getenv('HUGGINGFACEHUB_API_TOKEN')
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings()

Created a chunk of size 11806, which is longer than the specified 1000
Created a chunk of size 1139, which is longer than the specified 1000
Created a chunk of size 1211, which is longer than the specified 1000
Created a chunk of size 2934, which is longer than the specified 1000
Created a chunk of size 1552, which is longer than the specified 1000
Created a chunk of size 1139, which is longer than the specified 1000
Created a chunk of size 1211, which is longer than the specified 1000
Created a chunk of size 2934, which is longer than the specified 1000
Created a chunk of size 1552, which is longer than the specified 1000
Created a chunk of size 1250, which is longer than the specified 1000
Created a chunk of size 1770, which is longer than the specified 1000
Created a chunk of size 1263, which is longer than the specified 1000
Created a chunk of size 1404, which is longer than the specified 1000
Created a chunk of size 3872, which is longer than the specified 1000
Created a chunk of 

In [6]:
db = Chroma.from_documents(texts, embeddings, collection_name="hubspot_hubl_knowledge_base")

In [7]:
from langchain.chat_models import ChatLiteLLM
from langchain.prompts.chat import (
    ChatPromptTemplate
)

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()
from langchain.llms import Cohere
cohere_api_key= os.getenv('COHERE_API_KEY')
cohere_llm = ChatLiteLLM(model="command-nightly")


In [9]:
import os
from dotenv import load_dotenv

load_dotenv()
google_api_key= os.getenv('GOOGLE_API_KEY')
google_palm_llm = ChatLiteLLM(model="palm/chat-bison")

In [10]:
hubl_docs_chain = RetrievalQA.from_chain_type(
    llm=cohere_llm, chain_type="stuff", retriever=db.as_retriever()
)

In [11]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.utilities import PythonREPL
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import DuckDuckGoSearchRun 
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.tools import ShellTool
from langchain.agents import load_tools

In [12]:
#DuckDuckGo Class
duckduck_search = DuckDuckGoSearchRun()
shell_tool = ShellTool()


#Python REPL Class
python_repl = PythonREPL()

shell_tool = Tool(
    name="shell",
    description="This tool gives you access to a system's shell. Can be used to run shell commands",
    func=shell_tool.run
)


#DuckDuckGo Tool
duckduck_tool=Tool(
        name = "DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
)

#Python REPL Agent Tool Class
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run
)

In [13]:
tools = [
    Tool(
        name="HubL Docs",
        func=hubl_docs_chain.run,
        description="Useful for when you need to answer questions about HubL. output should be code",
    ),
    Tool(
        name = "DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
    name="shell",
    description="This tool gives you access to a system's shell. Can be used to run shell commands",
    func=shell_tool.run
    )
]

In [14]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, google_palm_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
)

In [16]:
agent.run(
    "What is HubL?"
)



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: HubL is a templating language used to create and customize content on HubSpot. It is a simple, yet powerful language that allows you to create dynamic and engaging content. HubL is based on JSON, so it is easy to learn and use.

To use HubL, you first need to create a HubSpot account. Once you have created an account, you can access the HubL documentation. The documentation includes a tutorial that will teach you the basics of HubL.

Once you have learned the basics of HubL, you can start using it to create your own content. To do this, you will need to create a template. A template is a file that contains HubL code. You can create a template in any text editor.

Once you have created a template, you can use it to create conte